In [9]:
import torch
from torchvision import datasets
from torchvision import transforms
import torch.nn as nn
import torch.optim as optim

### data

In [10]:
train_data = datasets.MNIST(root="data/mnist", train=True, transform=transforms.ToTensor(), download=True)
test_data = datasets.MNIST(root="data/mnist", train=False, transform=transforms.ToTensor(), download=True)

In [27]:
batch_size = 100
train_loader = torch.utils.data.DataLoader(dataset=train_data,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data,batch_size=batch_size,shuffle=False)

### net

In [28]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [29]:
input_size = 28 * 28
hidden_size = 512
num_classes = 10

model = MLP(input_size, hidden_size, num_classes)

In [30]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### train

In [33]:
num_epochs = 2
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28 * 28)
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

Epoch [1/2], Step [100/600], Loss: 0.0216
Epoch [1/2], Step [200/600], Loss: 0.0257
Epoch [1/2], Step [300/600], Loss: 0.0452
Epoch [1/2], Step [400/600], Loss: 0.0052
Epoch [1/2], Step [500/600], Loss: 0.0033
Epoch [1/2], Step [600/600], Loss: 0.0129
Epoch [2/2], Step [100/600], Loss: 0.0195
Epoch [2/2], Step [200/600], Loss: 0.0001
Epoch [2/2], Step [300/600], Loss: 0.0149
Epoch [2/2], Step [400/600], Loss: 0.0363
Epoch [2/2], Step [500/600], Loss: 0.0283
Epoch [2/2], Step [600/600], Loss: 0.0343


### test

In [39]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28 * 28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')

Accuracy of the network on the 10000 test images: 98.25 %
